In [1]:
import bdt

bdt.auth("../poc.lic")
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

TypeError: 'JavaPackage' object is not callable

In [2]:
spark.sql("show functions like 'st_*'")

function
st_area
st_asgeojson
st_ashex
st_asjson
st_asqr
st_asqrclip
st_asqrprune
st_asqrspill
st_asrc
st_asrcclip


In [5]:
df = spark.sql('SELECT ST_FromXY(1.1, 2.2) SHAPE')
df.show(truncate=False)

+-------------------------------------------+
|SHAPE                                      |
+-------------------------------------------+
|{   �������?������@, 1.1, 2.2, 1.1, 2.2}|
+-------------------------------------------+



In [6]:
df\
.selectExpr('ST_X(SHAPE) X', 'ST_Y(SHAPE) Y')

X,Y
1.1,2.2


In [2]:
from bdt import functions as F
from pyspark.sql.functions import *

In [9]:
df.select(
    F.st_x(col('SHAPE')), 
    F.st_y(col('SHAPE'))
)

stx(SHAPE),sty(SHAPE)
1.1,2.2


In [3]:
sql("""
    SELECT
        ST_IsValid(ST_FromText('LINESTRING(0 0, 1 1)'), 4326) As `good line`,
        ST_IsValid(ST_FromText('POLYGON((0 0, 1 1, 1 2, 1 1, 0 0))'), 4326) As `bad poly`
""")

good line,bad poly
true,false


In [21]:
sql("""
    SELECT
        ST_IsValid(ST_FromText('LINESTRING(0 0, 1 1)'), wkid) As `good line`,
        ST_IsValid(ST_FromText('POLYGON((0 0, 1 1, 1 2, 1 1, 0 0))'), wkid) As `bad poly`
    FROM (SELECT 4326 as wkid)
""")

good line,bad poly
true,false


In [2]:
sql("""
SELECT ST_IsEmpty(ST_FromText('MULTIPOLYGON EMPTY'))
""")

stisempty(ST_FromText(MULTIPOLYGON EMPTY))
true


In [6]:
sql("""
SELECT ST_IsEmpty(ST_FromText('MULTIPOLYGON EMPTY'))
""")

stisempty(ST_FromText(MULTIPOLYGON EMPTY))
true


In [17]:
# https://stackoverflow.com/questions/34660867/convert-null-values-to-empty-array-in-spark-dataframe
sql("""
with t as (SELECT CAST(arr as array<binary>) FROM (SELECT array() arr))
SELECT size(arr) FROM t
""")

size(arr)
0


In [23]:
sql("""
with t as (SELECT CAST(array() as array<binary>) arr)
SELECT * FROM t
""")

arr
[]


In [7]:
spark\
.read\
.format('SHP')\
.option('path','/Users/kyun4731/Downloads/North_American_Roads/North_American_Roads.shp')\
.option('format','WKB')\
.load()\
.filter('length(shape) > 0')\
.selectExpr('ST_FromWKB(shape) SHAPE')\
.write\
.mode('Overwrite')\
.parquet('/tmp/deleteme')